In [ ]:
# --- Imports ---
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from pathlib import Path
import openpyxl

In [ ]:
# --- Paths (edit these if needed) ---
INPUT_PATH = Path(r"C:\Users\marcu\Documents\Marcus\ITDPA3-34\Project\The impact of Digital Media Consumption on Academic Performance.xlsx")
OUTPUT_XLSX = Path(r"C:\Users\marcu\Documents\Marcus\ITDPA3-34\Project\ITDPA3-34\dataset_encoded.xlsx")

In [ ]:
# --- Load ---
df_raw = pd.read_excel(INPUT_PATH)

In [ ]:
# --- Select columns of interest (match your survey) ---
cols = [
    "ID",
    "Which year of study are you currently in?",
    "Which category best fits your field of study?",
    "Which type of a higher education institution do you currently attend?",
    "What type of device do you mainly use for accessing digital media?",
    "Roughly how many hours per day do you spend on streaming platforms (Netflix, YouTube)?",
    "Roughly how many hours per day do you spend on Social media platforms (TikTok, Instagram, Whatsapp)?",
    "Which digital media platform do you use most frequently?\n",
    "What is your primary reason for using these platforms?\n",
    "How often do you encounter multitasking such as using apps while studying?",
    "How often do you binge-watch videos or series?",
    "Which platforms do you feel help you the most academically? (Select all that apply)",
    "Which platforms do you feel distract you the most from academics? (Select all that apply)",
    "What's your most recent academic performance average?",
    "How would you rate your overall academic performance this semester?\n",
    "How significantly does digital media distract you from your studies?",
    "How often has digital media usage negatively affected your exam preparation or assignment completion?",
    "How often do you use digital media platforms (like YouTube or TikTok) for academic assistance or study support?",
    "Overall, how would you describe the effect of digital media on your academic results?",
    "Which method do you use most to manage digital distractions during study sessions?",
    "If you could reduce or better manage your digital media use, do you think your academic performance would:",
    "Which view best aligns with your perception of the impact of digital media on academics?"
]

In [ ]:
df = df_raw[cols].copy()

In [ ]:
# --- Rename to concise snake_case ---
df = df.rename(columns={
    "Which year of study are you currently in?": "year",
    "Which category best fits your field of study?": "field",
    "Which type of a higher education institution do you currently attend?": "institution",
    "What type of device do you mainly use for accessing digital media?": "device",
    "Roughly how many hours per day do you spend on streaming platforms (Netflix, YouTube)?": "streamingHours",
    "Roughly how many hours per day do you spend on Social media platforms (TikTok, Instagram, Whatsapp)?": "socialHours",
    "Which digital media platform do you use most frequently?\n": "frequentPlatform",
    "What is your primary reason for using these platforms?\n": "primaryReason",
    "How often do you encounter multitasking such as using apps while studying?": "multitasking",
    "How often do you binge-watch videos or series?": "binge",
    "Which platforms do you feel help you the most academically? (Select all that apply)": "academicPlatform",
    "Which platforms do you feel distract you the most from academics? (Select all that apply)": "academicDistraction",
    "What's your most recent academic performance average?": "academicAvg",
    "How would you rate your overall academic performance this semester?\n": "academicPerformance",
    "How significantly does digital media distract you from your studies?": "digitalDistraction",
    "How often has digital media usage negatively affected your exam preparation or assignment completion?": "negativeEffect",
    "How often do you use digital media platforms (like YouTube or TikTok) for academic assistance or study support?": "academicSupport",
    "Overall, how would you describe the effect of digital media on your academic results?": "digitalEffect",
    "Which method do you use most to manage digital distractions during study sessions?": "distractionMethod",
    "If you could reduce or better manage your digital media use, do you think your academic performance would:": "manageDigital",
    "Which view best aligns with your perception of the impact of digital media on academics?": "perception",
})

In [ ]:
# --- Basic cleaning ---
# drop rows with no device (as in your earlier script)
df = df[df["device"].notna()].copy()

In [ ]:
# strip whitespace from all string cells to avoid mapping mismatches
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
# --- Ordinal/centroid mappings (become real numbers) ---
# Hours buckets -> numeric centroids (in hours)
hours_map = {
    "Less than 1 hour": 0.5,
    "1 - 2 hours": 1.5,
    "3 - 4 hours": 3.5,
    "5 - 6 hours": 5.5,
    "More than 6 hours": 6.5
}

In [ ]:
# Likert scales -> ordered numeric
likert_5_map = {
    "Never": 1, "Rarely": 2, "Sometimes": 3, "Often": 4, "Always": 5
}
likert_4_map = {  # for 4-point items you used
    "Never": 1, "Rarely": 2, "Occasionally": 3, "Regularly": 4
}
impact_5_map = {  # positive/negative effect
    "Very negative": 1, "Somewhat negative": 2, "Neutral (no significant impact)": 3,
    "Somewhat positive": 4, "Very positive": 5
}
distraction_5_map = {  # degree of distraction
    "Not at all": 1, "Rarely": 2, "Slightly": 3, "Moderately": 4, "Significantly": 5
}
neg_effect_4_map = {  # frequency of negative effect
    "Never": 1, "Rarely": 2, "Sometimes": 3, "Regularly": 4
}
support_4_map = {  # academic support frequency
    "Never": 1, "Rarely": 2, "Occasionally": 3, "Regularly": 4
}
performance_5_map = {  # self-rated performance
    "Poor": 1, "Below average": 2, "Average": 3, "Good": 4, "Excellent": 5
}
manage_map = {  # perceived impact of managing digital use
    "Get worse": 1, "Stay the same": 2, "Improve slightly": 3, "Improve Significantly": 4
}
# Academic average -> midpoints (in %)
avg_map = {
    "Below 50%": 45,
    "50 - 59 %": 55,
    "60 - 69 %": 65,
    "70 - 79 %": 75,  # ensure matches your survey label
    "80 % or higher": 85  # or 90; midpoint assumption
}

In [ ]:
# Year -> ordinal numeric
year_map = {"First Year": 1, "Second Year": 2, "Third Year": 3, "Fourth Year or Above": 4}

In [ ]:
# --- Apply ordinal mappings safely ---
def map_series(s: pd.Series, mapping: dict, dtype=float):
    return s.map(mapping).astype(dtype, errors="ignore")

In [ ]:
df["year_num"] = map_series(df["year"], year_map, int)
df["streamingHours_num"] = map_series(df["streamingHours"], hours_map, float)
df["socialHours_num"] = map_series(df["socialHours"], hours_map, float)
df["binge_num"] = map_series(df["binge"], likert_4_map, int)
df["multitasking_num"] = map_series(df["multitasking"], likert_5_map, int)
df["academicAvg_num"] = map_series(df["academicAvg"], avg_map, float)
df["academicPerformance_num"] = map_series(df["academicPerformance"], performance_5_map, int)
df["digitalDistraction_num"] = map_series(df["digitalDistraction"], distraction_5_map, int)
df["negativeEffect_num"] = map_series(df["negativeEffect"], neg_effect_4_map, int)
df["academicSupport_num"] = map_series(df["academicSupport"], support_4_map, int)
df["digitalEffect_num"] = map_series(df["digitalEffect"], impact_5_map, int)
df["manageDigital_num"] = map_series(df["manageDigital"], manage_map, int)

In [ ]:
# --- One-Hot Encode purely nominal columns ---
nominal_cols = [
    "field", "institution", "device", "frequentPlatform",
    "primaryReason", "distractionMethod", "perception"
]
df_nominal = pd.get_dummies(df[nominal_cols], prefix=nominal_cols, drop_first=True)

In [ ]:
# --- Multi-select column: academicDistraction (split on ';') ---
distraction_list = (
    df["academicDistraction"]
    .fillna("")
    .astype(str)
    .apply(lambda x: [i.strip() for i in x.split(";") if i.strip() != ""])
)
mlb = MultiLabelBinarizer()
if len(distraction_list) > 0:
    distraction_flags = pd.DataFrame(
        mlb.fit_transform(distraction_list),
        columns=[f"distraction_{c}" for c in mlb.classes_],
        index=df.index
    )
else:
    distraction_flags = pd.DataFrame(index=df.index)

In [ ]:
# --- Assemble encoded dataset ---
numeric_cols = [
    "year_num", "streamingHours_num", "socialHours_num",
    "binge_num", "multitasking_num",
    "academicAvg_num", "academicPerformance_num",
    "digitalDistraction_num", "negativeEffect_num",
    "academicSupport_num", "digitalEffect_num", "manageDigital_num"
]

base_cols_to_keep = ["ID"]  # keep identifiers you need; add others if required

In [ ]:
df_encoded = pd.concat(
    [df[base_cols_to_keep], df_nominal, distraction_flags, df[numeric_cols]],
    axis=1
)

In [ ]:
# --- Enforce numeric dtypes where applicable ---
for c in numeric_cols:
    df_encoded[c] = pd.to_numeric(df_encoded[c], errors="coerce")

# Optional: drop rows with any critical missing numeric targets/predictors
# df_encoded = df_encoded.dropna(subset=["academicAvg_num", "academicPerformance_num"])

In [ ]:
# --- Save outputs ---
OUTPUT_XLSX.parent.mkdir(parents=True, exist_ok=True)
df_encoded.to_excel(OUTPUT_XLSX, index=False)

print(f"Saved encoded dataset:\n- {OUTPUT_XLSX}")
print(f"Shape: {df_encoded.shape}")

Saved encoded dataset:
- C:\Users\marcu\Documents\Marcus\ITDPA3-34\Project\ITDPA3-34\dataset_encoded.xlsx
Shape: (110, 44)
